In [11]:
import os,json
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten, Input, Conv1D,MaxPooling1D,Embedding
from keras.models import load_model, Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report
import pandas as pd
import sqlalchemy as sa
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter


user = 'dev_ds'
pwd = 'akBefvuIj4'
from base64 import b64decode
conn_str = 'mssql+pyodbc://%s:%s@repdb1irv.iherb.net/iHerb_Shop?driver=ODBC+Driver+13+for+SQL+Server'
engine = sa.create_engine(conn_str % ('dev_ds', b64decode('YWtCZWZ2dUlqNA==').decode()))

In [12]:
p='inventoryvelocity'
query3='''
SELECT A.ProductID as ProductID, A.ParentCategoryID as CategoryID, B.level as level, B.d as d, B.Name as Name 
FROM `inventoryvelocity.input.product_parent_categories_relation` AS A
Inner join `rec-serv.AutoSuggestion.category_data` AS B on A.ParentCategoryID=B.CategoryID 
Where B.VisibleToCustomer=True and B.CategoryID not in (1475,1855,2089,2203,2236,2320,2992,3213,100477,100483,2430,2493,2465)
'''
products_OldCategories=pd.read_gbq(query3,p,dialect='standard')

product_attr = pd.read_sql(''' 
SELECT [ProductID]
      ,[DisplayName]
      ,[Ingredients]
      ,[Description]
      ,[SupplementFacts]
      ,[ServingSize]
      ,[PackageQuantity]
      ,[SuggestedUse]
      ,[Keywords]
  FROM [iHerb_Shop].[shopservice].[TBL_Products]
  where Status in (0,3,4,6)
''', engine)
product_attr.set_index('ProductID' , inplace=True)
print(len(product_attr))
products_OldCategories=products_OldCategories[products_OldCategories['ProductID'].isin(product_attr.index.values)]
print(products_OldCategories['ProductID'].nunique())

30070
30069


In [13]:
query1='''
SELECT * FROM [rec-serv:AutoSuggestion.category_data] 
'''
category_data=pd.read_gbq(query1,p)
category_data.set_index('CategoryID' , inplace=True)



In [14]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0, stop_words='english')
tf.fit(product_attr['DisplayName'])
tfidf_matrix_allProducts=tf.transform(product_attr['DisplayName'])
cosine_similarities=np.zeros((tfidf_matrix_allProducts.shape[0],tfidf_matrix_allProducts.shape[0]))
cosine_similarities=np.asarray(cosine_similarity(tfidf_matrix_allProducts,tfidf_matrix_allProducts))

In [15]:
product_similarities=pd.DataFrame(cosine_similarities, index=product_attr.index.values.tolist(),columns=product_attr.index.values.tolist())
product_similar_dict = dict()
for product in product_attr.index.tolist():
    product_similar_dict[product] = product_similarities.loc[product].sort_values(ascending=False).index.tolist()[1:11]

In [16]:
print(products_OldCategories[products_OldCategories['level']==1].nunique())
products_level0=products_OldCategories[products_OldCategories['level']==0]
products_level1=products_OldCategories[products_OldCategories['level']==1]
products_level2=products_OldCategories[products_OldCategories['level']==2]
products_level3=products_OldCategories[products_OldCategories['level']==3]
products_level4=products_OldCategories[products_OldCategories['level']==4]
products_level5=products_OldCategories[products_OldCategories['level']==5]


ProductID     29536
CategoryID     1462
level             1
d              1462
Name           1462
dtype: int64


In [17]:
def find_category(p,products_level):
    if p in products_level['ProductID'].values.tolist():
        df=products_level[products_level['ProductID']==p]
        return df['CategoryID'].values.tolist()
    else:
        return [0]
    

In [18]:
extra_level0_products=[i for i in product_attr.index.values if i not in products_level0['ProductID'].values.tolist()]
extra_level1_products=[i for i in product_attr.index.values if i not in products_level1['ProductID'].values.tolist()]
extra_level2_products=[i for i in product_attr.index.values if i not in products_level2['ProductID'].values.tolist()]
extra_level3_products=[i for i in product_attr.index.values if i not in products_level3['ProductID'].values.tolist()]
extra_level4_products=[i for i in product_attr.index.values if i not in products_level4['ProductID'].values.tolist()]
extra_level5_products=[i for i in product_attr.index.values if i not in products_level5['ProductID'].values.tolist()]

In [9]:
for product in extra_level1_products:
    print(product)
    nested_categories= [find_category(x,products_level1) for x in product_similar_dict[product]]
    flat_categories = [item for sublist in nested_categories for item in sublist]
   # print(flat_categories)
    most_commons= Counter(flat_categories).most_common(5)
    print(most_commons)
    

64


NameError: name 'product_similar_dict' is not defined

In [19]:
file_category_results = open("ExtraProducts__category.txt","w")
count=0

extra_levels_products=[(extra_level1_products,products_level1),(extra_level2_products,products_level2),(extra_level3_products,products_level3),\
                       (extra_level4_products,products_level4),(extra_level5_products,products_level5)]
for level in extra_levels_products:
    for product in level[0]:
        nested_categories= [find_category(x,level[1]) for x in product_similar_dict[product]]
        flat_categories = [item for sublist in nested_categories for item in sublist]
       # print(flat_categories)
        most_commons= Counter(flat_categories).most_common(5)
        for m in most_commons:
            if m[1] > 5 and m[0]!=0:
                file_category_results.write("ProductID:{0}    CategoryID:{1}   Name:{2}              {3}\n".format(product,m[0],\
                category_data.loc[m[0]]['Name'],category_data.loc[m[0]]['d']))
                count+=1
file_category_results.close()

In [20]:
print(count)

873


In [21]:
file_category_results = open("Products_category_TFIDF_Cosine.txt","w")
count_all=0


for product in product_attr.index.values.tolist():
    nested_categories= [find_category(x,products_OldCategories) for x in product_similar_dict[product]]
    flat_categories = [item for sublist in nested_categories for item in sublist]
   # print(flat_categories)
    most_commons= Counter(flat_categories).most_common(10)
    for m in most_commons:
        if (m[1] > 6 and m[0]!=0 and ~(((products_OldCategories['ProductID']==product) & (products_OldCategories['CategoryID']==m[0])).any())):
            file_category_results.write("ProductID:{0}    CategoryID:{1}   Name:{2}              {3}\n".format(product,m[0],\
            category_data.loc[m[0]]['Name'],category_data.loc[m[0]]['d']))
            count_all+=1
file_category_results.close()

KeyboardInterrupt: 

In [15]:
query3='''
SELECT A.ProductID as ProductID, A.ParentCategoryID as CategoryID, B.level as level, B.d as d, B.Name as Name 
FROM `inventoryvelocity.input.product_parent_categories_relation` AS A
Inner join `rec-serv.AutoSuggestion.category_data` AS B on A.ParentCategoryID=B.CategoryID
WHERE B.CategoryID not in (1791,1738,1775)
and B.ParentCategoryID in (101072)
'''
products_vitamins=pd.read_gbq(query3,p,dialect='standard')

products_vitamins=products_vitamins[products_vitamins['ProductID'].isin(product_attr.index.values)]

In [16]:
print(product_similar_dict[8422])
nested_categories= [find_category(x,products_vitamins) for x in product_similar_dict[8422]]

[48609, 17417, 1308, 3950, 50076, 37670, 73747, 32551, 80889, 59913]
